## Import Libraries and get stockoverflow library set up to make api requests

In [1]:
import stackexchange
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

api_key = '5*wCDZgONnIJ*aiaYeKjQQ(('

so = stackexchange.Site(stackexchange.StackOverflow,api_key)
so.be_inclusive()
so.impose_throttling = True
so.throttle_stop = False

## Looking at a specific question

In [369]:
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

p_stemmer = PorterStemmer()
tokenizer = RegexpTokenizer('\w+')

class acquire_SO_question_info:
    def __init__(self,q_id, search=True):
        if search == True:
            self.question = so.question(q_id)
            self.id = q_id
        else:
            self.question = q_id
            self.id = q_id.id
            
        self.tokens = []
        self.raw = []
        self.score = 0
        self.view_count = question.view_count
        self.url = question.link
        
    def fetch_and_clean(self,text):
        raw = BeautifulSoup(text, "lxml").get_text()
        raw = raw.lower()
        self.raw += [raw]
        tokens = tokenizer.tokenize(raw)

        tokens = [word for word in tokens if not word.isdigit()]
        tokens = [word for word in tokens if word not in stopwords.words('english')]
        tokens = [p_stemmer.stem(i) for i in tokens]
        return tokens       
    
    def get_Info(self,text_type):
        text_match = text_type
        
        if text_type.lower() == 'question':
            text_source = self.question
            text_match = None
        else:
            text_source = self.question.json
            
            
        if text_match == None:
            self.tokens += self.fetch_and_clean(text_source.body)
            self.score += text_source.score
        elif text_match in text_source.keys():
            for text in text_source[text_match]:
                self.tokens += self.fetch_and_clean(text['body'])
                self.score += text['score']
                if 'comments' in text.keys():
                    for comments in text['comments']:
                        self.tokens += self.fetch_and_clean(comments['body'])
                        self.score += comments['score']
        return self.tokens
    
    def get_sources(self):
        self.get_Info('question')
        self.get_Info('comments')
        self.get_Info('answers')
        pass
    
    def get_df_data(self):
        return [self.id, self.score,self.view_count,self.url]

In [373]:
rawtext_dict = {}
text_dict = {}
data = []

Data_file = open('Python_StackOverflow_Data.csv', 'w')
Txt_file = open('Python_StackOverflow_Txt.csv', 'w')
#Raw_file = open('Python_StackOverflow_RawTxt.csv', 'w')

for i,question in enumerate(so.questions(tagged=['python'], pagesize=10)):
    StackObj = acquire_SO_question_info(question,search=False)
    StackObj.get_sources()

    #text_dict[StackObj.id] = StackObj.tokens #text to dictionary
    #data.append(StackObj.get_df_data())
    #rawtext_dict[StackObj.id] = StackObj.raw

    Data_file.write("%d,%d,%d,%s\n" % (StackObj.id, StackObj.score, StackObj.view_count, StackObj.url))
    Txt_file.write("%d,%s\n" % (StackObj.id, StackObj.tokens))
    #Raw_file.write("%d,%s\n" % (StackObj.id, StackObj.raw))

    if i % 1000==0: print(i)
    #if i == 3: break

Data_file.close()
Data_file.close()

some of this might be out of data so check out the python file for downloading data from stackoverflow